In [60]:
import pandas as pd
import torch
import torchvision

import torch.nn as nn
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset

from PIL import Image

import os

In [36]:
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.ToTensor()])
train_dir = 'train'

In [96]:
class ReadData(Dataset):
    def __init__(self, imager_dir, transoform):
        self.imager_dir = imager_dir
        self.read_dir()
        self.all_class = {'apple' : 0,
                         'envelope' : 1,
                         'house' : 2,
                         'icecream' : 3,
                         'ladder' : 4,
                         'tshirt' : 5}
        self.transform = transform
    
    def read_dir(self):
        directory = os.listdir(self.imager_dir)
        b = [[]]*len(directory)
        dictionary = dict(zip(directory,b))

        allImages = []
        for i in dictionary.keys():
            images = os.listdir(os.path.join(self.imager_dir, i))
            fullPathes = [os.path.join(self.imager_dir, i, j) for j in images]
            allImages += fullPathes
            dictionary[i] = fullPathes
        
        self.pathAllImages = allImages

    def __len__(self):
        return len(self.pathAllImages)

    def __getitem__(self, idx):
        image_path = self.pathAllImages[idx]
        image_pil = Image.open(image_path)
        image_pil = image_pil.convert('L')
        label = ''.join([i for i in image_path.split('\\')[-1].split('.')[0] if not i.isdigit()]).lower()
        if self.transform:
            image = self.transform(image_pil)
        return image, self.all_class[label]

In [104]:
train_data = ReadData(train_dir, transform)
data, y = train_data[55]
y

1

In [98]:
class Noiro(nn.Module):
    def __init__(self, num_classes):
        super(Noiro, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5)
        self.conv_layer2 = nn.Conv2d(in_channels=12, out_channels=36, kernel_size=3)
        self.conv_layer3 = nn.Conv2d(in_channels=36, out_channels=36, kernel_size=5)
        self.conv_layer4 = nn.Conv2d(in_channels=36, out_channels=64, kernel_size=5)
        
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer2 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer3 = nn.MaxPool2d(kernel_size=2)
        
        self.linear_layer1 = nn.Linear(in_features=64, out_features=32)
        self.linear_layer2 = nn.Linear(in_features=32, out_features=6)
        
        self.relu = nn.ReLU()
        self.soft_max = nn.Softmax()
        
    def forward(self, inputs):
        output_1 = self.relu(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.relu(self.conv_layer3(output_4))
        output_6 = self.pooling_layer3(output_5)
        output_7 = self.relu(self.conv_layer4(output_6))
        
        output_8 = torch.flatten(output_7, 1)
        
        output_9 = self.relu(self.linear_layer1(output_8))
        output = self.soft_max(self.linear_layer2(output_9))
        
        return output

In [99]:
model = Noiro(6)
print(model)

Noiro(
  (conv_layer1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv_layer2): Conv2d(12, 36, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer3): Conv2d(36, 36, kernel_size=(5, 5), stride=(1, 1))
  (conv_layer4): Conv2d(36, 64, kernel_size=(5, 5), stride=(1, 1))
  (pooling_layer1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pooling_layer2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pooling_layer3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer1): Linear(in_features=64, out_features=32, bias=True)
  (linear_layer2): Linear(in_features=32, out_features=6, bias=True)
  (relu): ReLU()
  (soft_max): Softmax(dim=None)
)


In [100]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [101]:
train_data_loader = DataLoader(
    dataset=train_data,
    shuffle=True)

test_data_loader = DataLoader(
    dataset=ReadData('Val', transform),
    shuffle=True)

In [102]:
torch.manual_seed(0)

total_step = len(train_data_loader)
for epoch in range(5):
    for i, (images, labels) in enumerate(train_data_loader): 
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 60 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

<ipython-input-98-ef093bff9db8>:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.soft_max(self.linear_layer2(output_9))


Epoch [1/5], Step [60/180], Loss: 1.7610
Epoch [1/5], Step [120/180], Loss: 1.8734
Epoch [1/5], Step [180/180], Loss: 1.7367
Epoch [2/5], Step [60/180], Loss: 1.7766
Epoch [2/5], Step [120/180], Loss: 1.7833
Epoch [2/5], Step [180/180], Loss: 1.8246
Epoch [3/5], Step [60/180], Loss: 1.8417
Epoch [3/5], Step [120/180], Loss: 1.7258
Epoch [3/5], Step [180/180], Loss: 1.6947
Epoch [4/5], Step [60/180], Loss: 1.9825
Epoch [4/5], Step [120/180], Loss: 1.0493
Epoch [4/5], Step [180/180], Loss: 2.0101
Epoch [5/5], Step [60/180], Loss: 1.0436
Epoch [5/5], Step [120/180], Loss: 1.8242
Epoch [5/5], Step [180/180], Loss: 1.0447


In [103]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_data_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 90.0 %


<ipython-input-98-ef093bff9db8>:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.soft_max(self.linear_layer2(output_9))
